In [2]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.api as sm

In [3]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [4]:
funding_liquidity, market_liquidity = liquidity_observables()

tsmom_model.py:412: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['TED Spread']=ted.resample(rule='m',how='last')/100.
tsmom_model.py:413: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  funding_liquidity['LIBOR term repo']=libor_term_repo.resample(rule='m',how='last')/100.
tsmom_model.py:417: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  PS=pd.read_csv('PS.csv',index_col=0,parse_dates=True).resample(rule='m',how='last')
tsmom_model.py:418: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  us10y_zero=quandl.get('FED/SVENY',authtoken=token).SVENY10.resample(rule='d',how='last').dropna()
tsmom_model.py:419: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  yield10=quandl.get('USTREASURY/YIELD',authtoken=token)['10 YR'].resample(rule='d',how='last').dropna()
tsmom_

In [8]:
factors=get_all_factors()

In [21]:
results_df=pd.DataFrame()
for reg_fact in funding_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=funding_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
#results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [22]:
results_df.round(2).T

,0,1,2,3
TED Spread,-0.23,-1.17,-0.34,-0.76
TED SpreadTstats,-0.63,-2.69,-0.95,-0.95
LIBOR term repo,0.26,0.09,0.25,0.38
LIBOR term repoTstats,2.04,1.33,1.96,1.69


In [25]:
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [32]:
results_df.T.columns

Index([u'Amihud Annual', u'FHT Annual', u'Amihud Monthly', u'FHT Monthly'], dtype='object')

In [34]:
print results_df.T[[u'Amihud Annual', u'Amihud Monthly', u'FHT Annual', u'FHT Monthly']].round(2).to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  Amihud Annual &  Amihud Monthly &  FHT Annual &  FHT Monthly \\
\midrule
TED Spread                             &          -0.23 &           -0.34 &       -1.17 &        -0.76 \\
TED SpreadTstats                       &          -0.63 &           -0.95 &       -2.69 &        -0.95 \\
LIBOR term repo                        &           0.26 &            0.25 &        0.09 &         0.38 \\
LIBOR term repoTstats                  &           2.04 &            1.96 &        1.33 &         1.69 \\
PS                                     &           0.01 &            0.01 &        0.01 &         0.01 \\
PSTstats                               &           0.54 &            0.58 &        0.47 &         0.34 \\
On versus off the run Treasuries       &           0.02 &            0.01 &       -0.03 &        -0.03 \\
On versus off the run TreasuriesTstats &           0.75 &            0.59 &       -1.44 &        -1.56 \\
\bottomrule
\end{tabular}

